# Example using EM
Expectation maximization (EM) helps generate the gaussian mixture guesses based on a guess for the size of the object. This assumes that the user already has an Observation. We will start by creating our fake observation from the Basic_fitting notebook.